In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
torch.cuda.is_available()

True

In [12]:
num_epochs = 15
num_classes = 10
batch_size = 128
learning_rate = 0.001

In [13]:
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

#Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

100%|█████████▉| 9912320/9912422 [10:01<00:00, 12905.34it/s]Extracting ../../data/MNIST\raw\train-images-idx3-ubyte.gz to ../../data/MNIST\raw

0it [00:00, ?it/s]
 85%|████████▌ | 24576/28881 [00:05<00:00, 12852.54it/s]Extracting ../../data/MNIST\raw\train-labels-idx1-ubyte.gz to ../../data/MNIST\raw


0it [00:00, ?it/s]

  0%|          | 0/1648877 [00:02<?, ?it/s]

  0%|          | 8192/1648877 [00:02<01:11, 22977.57it/s]

  2%|▏         | 32768/1648877 [00:02<00:55, 29134.96it/s]

  2%|▏         | 40960/1648877 [00:03<00:56, 28708.70it/s]

  3%|▎         | 57344/1648877 [00:05<01:35, 16668.69it/s]

  4%|▍         | 65536/1648877 [00:05<01:22, 19265.18it/s]

  4%|▍         | 73728/1648877 [00:07<02:39, 9879.66it/s]

  5%|▌         | 90112/1648877 [00:07<02:15, 11471.37it/s]

  6%|▌         | 98304/1648877 [00:08<02:24, 10764.15it/s]

9920512it [10:17, 12905.34it/s]

  7%|▋         | 114688/1648877 [00:09<02:03, 12415.52it/s]

  7%|▋         | 122880/1648877 [00:10<01:41, 15022.88it/s]

In [14]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dense1 = nn.Linear(12*12*64, 128)
        self.dense2 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.dropout(x, 0.25)
        x = x.view(-1, 12*12*64)
        x = F.relu(self.dense1(x))
        x = F.dropout(x, 0.5)
        x = self.dense2(x)
        return F.log_softmax(x, dim = 1)

In [18]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
total_step = len(train_loader)
print(model)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dense1): Linear(in_features=9216, out_features=128, bias=True)
  (dense2): Linear(in_features=128, out_features=10, bias=True)
)


In [19]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch[{}/{}], Step[{}/{}], Loss:{:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch[1/15], Step[100/469], Loss:0.4211
Epoch[1/15], Step[200/469], Loss:0.1153
Epoch[1/15], Step[300/469], Loss:0.1231
Epoch[1/15], Step[400/469], Loss:0.1589
Epoch[2/15], Step[100/469], Loss:0.1272
Epoch[2/15], Step[200/469], Loss:0.1699
Epoch[2/15], Step[300/469], Loss:0.2045
Epoch[2/15], Step[400/469], Loss:0.0655
Epoch[3/15], Step[100/469], Loss:0.0392
Epoch[3/15], Step[200/469], Loss:0.0685
Epoch[3/15], Step[300/469], Loss:0.0422
Epoch[3/15], Step[400/469], Loss:0.0998
Epoch[4/15], Step[100/469], Loss:0.0410
Epoch[4/15], Step[200/469], Loss:0.0979
Epoch[4/15], Step[300/469], Loss:0.0166
Epoch[4/15], Step[400/469], Loss:0.0319
Epoch[5/15], Step[100/469], Loss:0.0387
Epoch[5/15], Step[200/469], Loss:0.0499
Epoch[5/15], Step[300/469], Loss:0.0479
Epoch[5/15], Step[400/469], Loss:0.0348
Epoch[6/15], Step[100/469], Loss:0.0746
Epoch[6/15], Step[200/469], Loss:0.0455
Epoch[6/15], Step[300/469], Loss:0.1199
Epoch[6/15], Step[400/469], Loss:0.0685
Epoch[7/15], Step[100/469], Loss:0.0396


In [20]:
from sklearn.metrics import roc_auc_score

preds = []
y_true = []
# Test the model
model.eval()  # Set model to evaluation mode.
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()    
        detached_pred = predicted.detach().cpu().numpy()
        detached_label = labels.detach().cpu().numpy()
        for f in range(0, len(detached_pred)):
            preds.append(detached_pred[f])
            y_true.append(detached_label[f])
        
    print('Test Accuracy of the model on the 10000 test images: {:.2%}'.format(correct / total))
    
    preds = np.eye(num_classes)[preds]
    y_true = np.eye(num_classes)[y_true]    
    auc = roc_auc_score(preds, y_true)
    print("AUC: {:.2%}".format (auc))
# Save the model checkpoint
torch.save(model.state_dict(), 'pytorch_mnist_cnn.ckpt')

Test Accuracy of the model on the 10000 test images: 98.37%
AUC: 99.09%


In [21]:
with torch.no_grad():
    correct = 0
    total = 0
    predss = []
    matches = []
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        for f in range(0, len(predicted.detach().cpu().numpy())):
            predss.append(predicted.detach().cpu().numpy()[f])
            matches.append(labels.detach().cpu().numpy()[f])
            
#     preds = np.eye(num_classes)[predicted.detach().cpu().numpy()]
#     y_true = np.eye(num_classes)[labels.detach().cpu().numpy()]
#     print(predss, matches)
    preds = np.eye(num_classes)[predss]
    y_true = np.eye(num_classes)[matches]
    
    print(preds == y_true)
    auc = roc_auc_score(preds, y_true)
    print("AUC: {:.2%}".format (auc))

[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]
AUC: 99.14%


In [23]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    
    
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0),
#             nn.BatchNorm2d(16),
            nn.ReLU())
#             nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            
        
        )
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)
print(model)

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential()
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)


## nn.Sequential(Recommended)
一般情况下 nn.Sequential 的用法是来组成卷积块 (block)，然后像拼积木一样把不同的 block 拼成整个网络，让代码更简洁，更加结构化。

In [24]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv = nn.Sequential(         # A sequential container
            nn.Conv2d(3, 32, 3, 1, 1),     # nn.Sequential里面的模块按照顺序进行排列的，所以必须确保前一个模块的输出大小和下一个模块的输入大小是一致的
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.dense = nn.Sequential(
            nn.Linear(32 * 3 * 3, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        conv_out = self.conv1(x)
        res = conv_out.view(conv_out.size(0), -1)
        out = self.dense(res)
        return out

model2 = Net2()
print(model2)

Net2(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=288, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)


## nn.ModuleList
是一个储存不同 module，并自动将每个 module 的 parameters 添加到网络之中的容器。你可以把任意 nn.Module 的子类 (比如 nn.Conv2d, nn.Linear 之类的) 加到这个 list 里面，方法和 Python 自带的 list 一样，无非是 extend，append 等操作。但不同于一般的 list，加入到 nn.ModuleList 里面的 module 是会自动注册到整个网络上的，同时 module 的 parameters 也会自动添加到整个网络中。若使用python的list，则会出问题。

In [8]:
class net_modlist(nn.Module):
    def __init__(self):
        super(net_modlist, self).__init__()
        self.modlist = nn.ModuleList([
                       nn.Conv2d(1, 20, 5),
                       nn.ReLU(),
                        nn.Conv2d(20, 64, 5),
                        nn.ReLU()
                        ])

    def forward(self, x):
        for m in self.modlist:
            x = m(x)
        return x

net_modlist = net_modlist()
print(net_modlist)
for param in net_modlist.parameters():
    print(type(param.data), param.size())

net_modlist(
  (modlist): ModuleList(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
    (3): ReLU()
  )
)
<class 'torch.Tensor'>torch.Size([20, 1, 5, 5])
<class 'torch.Tensor'>torch.Size([20])
<class 'torch.Tensor'>torch.Size([64, 20, 5, 5])
<class 'torch.Tensor'>torch.Size([64])


In [7]:
class net_modlist(nn.Module):
    def __init__(self):
        super(net_modlist, self).__init__()
        self.modlist = [
                       nn.Conv2d(1, 20, 5),
                       nn.ReLU(),
                        nn.Conv2d(20, 64, 5),
                        nn.ReLU()
                        ]

    def forward(self, x):
        for m in self.modlist:
            x = m(x)
        return x

net_modlist = net_modlist()
print(net_modlist)

net_modlist()


## nn.Sequential与nn.ModuleList的区别
### 1. nn.Sequential内部实现了forward函数，因此可以不用写forward函数。而nn.ModuleList则没有
### 2. nn.Sequential可以使用OrderedDict对每层进行命名
### 3. nn.Sequential里面的模块按照顺序进行排列的，所以必须确保前一个模块的输出大小和下一个模块的输入大小是一致的。而nn.ModuleList 并没有定义一个网络，它只是将不同的模块储存在一起，这些模块之间并没有什么先后顺序，网络的执行顺序是根据 forward 函数来决定的
### 4. nn.ModuleList有的时候网络中有很多相似或者重复的层，我们一般会考虑用for循环来创建它们，而不是一行一行地写

In [10]:
seq = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          nn.Conv2d(20,64,5),
          nn.ReLU()
        )
print(seq)

Sequential(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (3): ReLU()
)


In [13]:
input = torch.randn(16, 1, 20, 20)
print(seq(input).shape)

torch.Size([16, 64, 12, 12])


In [14]:
# 继承nn.Module类的话，就要写出forward函数
class net1(nn.Module):
    def __init__(self):
        super(net1, self).__init__()
        self.seq = nn.Sequential(
                        nn.Conv2d(1,20,5),
                         nn.ReLU(),
                          nn.Conv2d(20,64,5),
                       nn.ReLU()
                       )      
    def forward(self, x):
        return self.seq(x)

    #注意：按照下面这种利用for循环的方式也是可以得到同样结果的
    #def forward(self, x):
    #    for s in self.seq:
    #        x = s(x)
    #    return x

 #对net1进行输入
input = torch.randn(16, 1, 20, 20)
net1 = net1()
print(net1(input).shape)

torch.Size([16, 64, 12, 12])


In [17]:
# 不写forward报错
modlist = nn.ModuleList([
         nn.Conv2d(1, 20, 5),
         nn.ReLU(),
         nn.Conv2d(20, 64, 5),
         nn.ReLU()
         ])
print(modlist)

ModuleList(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (3): ReLU()
)


In [18]:
input = torch.randn(16, 1, 20, 20)
print(modlist(input).shape)

NotImplementedError: 

In [19]:
# 写出forward函数
class net2(nn.Module):
    def __init__(self):
        super(net2, self).__init__()
        self.modlist = nn.ModuleList([
                       nn.Conv2d(1, 20, 5),
                       nn.ReLU(),
                        nn.Conv2d(20, 64, 5),
                        nn.ReLU()
                        ])

    #这里若按照这种写法则会报NotImplementedError错
    #def forward(self, x):
    #    return self.modlist(x)

    #注意：只能按照下面利用for循环的方式
    def forward(self, x):
        for m in self.modlist:
            x = m(x)
        return x

input = torch.randn(16, 1, 20, 20)
net2 = net2()
print(net2(input).shape)

torch.Size([16, 64, 12, 12])


In [21]:
# 使用OrderedDict对每层进行命名
from collections import OrderedDict
model = nn.Sequential(OrderedDict([
    ('conv1', nn.Conv2d(1, 20, 5)),
    ('ReLU1', nn.ReLU()),
    ('conv2', nn.Conv2d(20, 64, 5)),
    ('ReLU2', nn.ReLU())
    ]))
print(model)

Sequential(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (ReLU1): ReLU()
  (conv2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (ReLU2): ReLU()
)


In [22]:
# nn.module不要求顺序
class net3(nn.Module):
    def __init__(self):
        super(net3, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10,20), nn.Linear(20,30), nn.Linear(5,10)])
    def forward(self, x):
        x = self.linears[2](x)
        x = self.linears[0](x)
        x = self.linears[1](x)

        return x

net3 = net3()
print(net3)

net3(
  (linears): ModuleList(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): Linear(in_features=20, out_features=30, bias=True)
    (2): Linear(in_features=5, out_features=10, bias=True)
  )
)


In [23]:
# nn.ModuleList批量生成网络
class net4(nn.Module):
    def __init__(self):
        super(net4, self).__init__()
        layers = [nn.Linear(10, 10) for i in range(5)]
        self.linears = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.linears:
            x = layer(x)
        return x

net = net4()
print(net)

net4(
  (linears): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=10, bias=True)
    (4): Linear(in_features=10, out_features=10, bias=True)
  )
)
